In [1]:
import os
import zipfile
import requests

import pandas as pd
import numpy as np
import fiona
import geopandas as gpd

from sqlalchemy import create_engine

In [2]:
response = requests.get('https://api.os.uk/downloads/v1/products')

In [3]:
products = pd.DataFrame(response.json())

In [4]:
products['download'] = products.url.apply(lambda x: requests.get(x).json()['downloadsUrl'])

In [25]:
products['download2'] = products.download.apply(lambda x: requests.get(x).json())

In [26]:
products['formats'] = products.download2.apply(lambda x: [e['format'] for e in x])

In [31]:
products['geopackage'] = products.formats.apply(lambda x: 'GeoPackage' in x)

In [51]:
products['geopackage_url'] = products.apply(lambda x: [e['url'] for e in x.download2 
                                                       if e['format'] == 'GeoPackage'][0] 
                                            if x.geopackage else None, axis=1)

In [55]:
def download_geopackage(row):
    print(row.id)
    with open(f'../data/{row.id}_gpkg.zip', 'wb') as downloading:
        downloading.write(requests.get(row.geopackage_url).content)
        downloading.close()
    return "Done!"

In [ ]:
products['downloaded'] = products.apply(
    lambda x: download_geopackage(x) if x.geopackage else None, axis=1
)
                                        